In [9]:
import networkx as nx
from tqdm import tqdm
import pandas as pd
from google.colab import auth
import numpy as np
import logging

auth.authenticate_user()
print('Authenticated')

Authenticated


# Exemplo de Busca Geral

In [6]:
%%bigquery --project gat-event-embedding df_temp
SELECT REGEXP_EXTRACT(Extras, '<PAGE_TITLE>(.*)</PAGE_TITLE>') text, * FROM `gdelt-bq.gdeltv2.gkg_partitioned` WHERE DATE(_PARTITIONTIME) = "2020-08-24" LIMIT 100

In [7]:
df_temp

,text,GKGRECORDID,DATE,SourceCollectionIdentifier,SourceCommonName,DocumentIdentifier,Counts,V2Counts,Themes,V2Themes,Locations,V2Locations,Persons,V2Persons,Organizations,V2Organizations,V2Tone,Dates,GCAM,SharingImage,RelatedImages,SocialImageEmbeds,SocialVideoEmbeds,Quotations,AllNames,Amounts,TranslationInfo,Extras
0,&#x62E;&#x64A;&#x628;&#x629; &#x627;&#x644;&#x...,20200824224500-T1560,20200824224500,1,albayan.ae,https://www.albayan.ae/sports/international/20...,None,None,None,None,1#Germany#GM#GM#51.5#10.5#GM;1#Qatar#QA#QA#25....,1#French#FR#FR##46#2#FR#1329;1#French#FR#FR##4...,bayer leverkusen;manuel neuer;kingsley koeman,"Bayer Leverkusen,561;Bayer Leverkusen,911;Manu...",champions league;twitter;center marquinhos,"Champions League,294;Twitter,616;Twitter,972;C...","-0.678733031674208,4.75113122171946,5.42986425...",None,"wc:420,nwc:449,c12.1:47,c12.10:42,c12.12:23,c1...",https://www.albayan.ae/polopoly_fs/1.3943892.1...,None,None,https://youtube.com/user/albayanonline;,None,"Star Brazilian Neymar,106;Champions Europe,167...","1000000000,a round years the,1948;",srclc:ara;eng:GT-ARA 1.0,<PAGE_AUTHORS>&#x627;&#x644;&#x628;&#x64A;&#x6...
1,&#x623;&#x62E;&#x628;&#x627;&#x631; 24 | &#x62...,20200824224500-T1566,20200824224500,1,argaam.com,https://akhbaar24.argaam.com/article/detail/50...,None,None,TAX_FNCACT;TAX_FNCACT_SPEAKER;TAX_FNCACT_OFFIC...,"TAX_FNCACT_SPEAKER,114;TAX_FNCACT_SPEAKER,798;...",1#Saudi Arabia#SA#SA#25#45#SA,1#Saudi Arabia#SA#SA##25#45#SA#262;1#Saudi Ara...,abdul rahman hussein,"Abdul Rahman Hussein,161",None,None,"-1.48148148148148,0.740740740740741,2.22222222...",None,"wc:125,nwc:176,c1.2:1,c12.1:5,c12.10:7,c12.12:...",https://s3-eu-west-1.amazonaws.com/content.arg...,None,https://pic.twitter.com/719lly2O5u;,https://youtube.com/channel/UC9Sbkq4oR_SIWMWEH...,None,"News Xxiv,10;Abdul Rahman Hussein,165;Xxv Augu...",None,srclc:ara;eng:GT-ARA 1.0,<PAGE_LINKS>https://t.co/719lly2O5u;https://tw...
2,&#x627;&#x644;&#x645;&#x648;&#x627;&#x631;&#x6...,20200824224500-T1488,20200824224500,1,iraqicp.com,https://www.iraqicp.com/index.php/sections/var...,None,None,UNGP_FORESTS_RIVERS_OCEANS;WB_137_WATER;AGRICU...,"GENERAL_GOVERNMENT,3840;EPU_POLICY_GOVERNMENT,...","4#Kirkuk, At Ta'mim, Iraq#IZ#IZ13#35.4681#44.3...","4#Darbandikhan, As Sulaymaniyah, Iraq#IZ#IZ05#...",gull hassan;ziraah hamid naev;aoun diab,"Gull Hassan,98;Ziraah Hamid Naev,2868;Aoun Dia...",ministry resources,"Ministry Resources,438;Ministry Resources,2212","-2.19966159052453,1.86125211505922,4.060913705...",None,"wc:572,nwc:650,c1.2:4,c12.1:28,c12.10:73,c12.1...",None,None,None,https://youtube.com/user/iraqicp/;,None,"Gull Hassan,101;Ministries Resources,133;Libra...","2,meters,778;",srclc:ara;eng:GT-ARA 1.0,<PAGE_AUTHORS>oscar</PAGE_AUTHORS><PAGE_PRECIS...
3,&#x643;&#x627;&#x645;&#x644; &#x634;&#x64A;&#x...,20200824224500-T1503,20200824224500,1,iraqicp.com,https://www.iraqicp.com/index.php/sections/lit...,None,None,KILL;ASSASSINATION;TAX_FNCACT;TAX_FNCACT_CHANC...,"BAN,2583;MANMADE_DISASTER_IMPLIED,2716;WB_696_...","4#Karbala, Karbala', Iraq#IZ#IZ12#32.5979#44.0...","4#Louvain, Region Flamande, Belgium#BE#BE13#58...",riham jacob;muhammad abdul orion;khazal majidi...,"Riham Jacob,3029;Riham Jacob,4628;Muhammad Abd...",center dc baghdad;agency associated;iraqi comm...,"Center Dc Baghdad,270;Agency Associated,2035;I...","1.10803324099723,5.81717451523546,4.7091412742...",None,"wc:653,nwc:772,c1.1:2,c1.3:1,c12.1:51,c12.10:6...",None,None,None,https://youtube.com/user/iraqicp/;,None,"Hussein Saleh,95;Xxiii August,229;Chancellor M...",None,srclc:ara;eng:GT-ARA 1.0,<PAGE_AUTHORS>oscar</PAGE_AUTHORS><PAGE_PRECIS...
4,&#x627;&#x644;&#x627;&#x62D;&#x62A;&#x62C;&#x6...,20200824224500-T1520,20200824224500,1,iraqicp.com,https://www.iraqicp.com/index.php/sections/orb...,"PROTEST#200#citizens#4#Baghdad, Baghdad, Iraq#...","PROTEST#200#citizens#4#Baghdad, Baghdad, Iraq#...",TAX_FNCACT;TAX_FNCACT_KILLERS;PROTEST;MOVEMENT...,"GENERAL_GOVERNMENT,2442;EPU_POLICY_GOVER

# Lista de Temas de Eventos

In [10]:
df_themes = pd.read_csv('http://data.gdeltproject.org/api/v2/guides/LOOKUP-GKGTHEMES.TXT',header=None,sep="\t")
df_themes

,0,1
0,TAX_FNCACT,779120478
1,TAX_ETHNICITY,324224345
2,EPU_POLICY,284818778
3,CRISISLEX_CRISISLEXREC,279041209
4,TAX_WORLDLANGUAGES,274229967
...,...,...
56835,TAX_WORLDLANGUAGES_LULUYIA,1
56836,TAX_WORLDBIRDS_FIVESTRIPED_SPARROWS,1
56837,TAX_DISEASE_GLUTAMATE_FORMIMINOTRANSFERASE_DEF...,1
56838,TAX_WORLDBIRDS_REDFRONTED_PARROTLET,1


In [11]:
themes = df_themes.iloc[:200]
themes

,0,1
0,TAX_FNCACT,779120478
1,TAX_ETHNICITY,324224345
2,EPU_POLICY,284818778
3,CRISISLEX_CRISISLEXREC,279041209
4,TAX_WORLDLANGUAGES,274229967
...,...,...
195,APPOINTMENT,23967038
196,WB_855_LABOR_MARKETS,23932623
197,CRISISLEX_O01_WEATHER,23588777
198,WB_2769_JOBS_STRATEGIES,23571400


# Exemplo de Pré-Processamento via Embeddings

In [3]:
!pip install git+https://github.com/rmarcacini/sentence-transformers
!pip install gdown
!gdown https://drive.google.com/uc?id=1NV5t1YhyyOzMF5zAovfbSLdZZLvqrfZ_
!unzip distiluse-base-multilingual-cased.zip -d language_model

  Cloning https://github.com/rmarcacini/sentence-transformers to /tmp/pip-req-build-s4sn8h1l
  Running command git clone -q https://github.com/rmarcacini/sentence-transformers /tmp/pip-req-build-s4sn8h1l
     |████████████████████████████████| 890kB 2.8MB/s 
     |████████████████████████████████| 3.0MB 17.9MB/s 
     |████████████████████████████████| 890kB 36.5MB/s 
     |████████████████████████████████| 1.1MB 28.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.6-cp36-none-any.whl size=101874 sha256=40ba978c28eda6bb7777fdcee2dd271f80014f2aed1b1200eddf81eddf280f6c
  Stored in directory: /tmp/pip-ephem-wheel-cache-hyjtiucj/wheels/88/3c/66/55ee9fb698480d5a5116a8257c15dc363323e4922fb8ad361b
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=d4cc97dab50a7aa838b161be9ebfa1508b69b72304f43ddf2e8ec682f477f6f3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f

In [12]:
from sentence_transformers import SentenceTransformer, LoggingHandler

np.set_printoptions(threshold=100)
logging.basicConfig(format='%(asctime)s - %(message)s',datefmt='%Y-%m-%d %H:%M:%S',level=logging.INFO,handlers=[LoggingHandler()])

language_model = SentenceTransformer('./language_model')

2020-10-29 12:21:28 - Load pretrained SentenceTransformer: ./language_model
2020-10-29 12:21:28 - Load SentenceTransformer from folder: ./language_model
2020-10-29 12:21:32 - Use pytorch device: cpu


In [13]:
df_temp['embedding'] = list(language_model.encode(df_temp['text'].to_list()))

In [14]:
# Removing rows with null "Themes" column
df_temp = df_temp.loc[~df_temp['Themes'].isna()]

In [15]:


G = nx.Graph()

for index,row in df_temp.iterrows():
    
    node_id = row['GKGRECORDID']
    node_date = str(row['DATE'])[0:8]
    node_themes_array = row['Themes'].split(';')
    node_locations_array = ''
    node_persons_array = ''
    node_organizations_array = ''

    try:
        node_locations_array = row['Locations'].split(';')
        node_persons_array = row['Persons'].split(';')
        node_organizations_array = row['Organizations'].split(';')
    except:
        1

    # event <-> date
    G.add_edge(node_id,node_date)
    G.nodes[node_id]['themes'] = node_themes_array
    G.nodes[node_date]['themes'] = node_themes_array
    
    # event <-> theme
    for theme in node_themes_array:
        if len(theme) > 0:
            G.add_edge(node_id,theme)
            G.nodes[theme]['themes'] = node_themes_array

    # event <-> locations
    for location in node_locations_array:
        if len(location) > 0:
            G.add_edge(node_id,location)
            G.nodes[location]['themes'] = node_themes_array


    # event <-> persons
    for person in node_persons_array:
        if len(person) > 0:
            G.add_edge(node_id,person)
            G.nodes[person]['themes'] = node_themes_array


    # event <-> organization
    for org in node_organizations_array:
        if len(org) > 0:
            G.add_edge(node_id,org)
            G.nodes[org]['themes'] = node_themes_array


    # embedding
    G.nodes[node_id]['embedding'] = row['embedding']
    
    print("node_id: \t", node_id)
    print("node_date: \t", node_date)
    print("node_themes_array: \t", node_themes_array)
    print("node_locations_array: \t", node_locations_array)
    print("node_persons_array: \t", node_persons_array)
    print("node_organizations_array: \t", node_organizations_array)
    print('----')
    

node_id: 	 20200824224500-T1566
node_date: 	 20200824
node_themes_array: 	 ['TAX_FNCACT', 'TAX_FNCACT_SPEAKER', 'TAX_FNCACT_OFFICIAL', 'WB_698_TRADE', 'TAX_FNCACT_CITIZENS', '']
node_locations_array: 	 ['1#Saudi Arabia#SA#SA#25#45#SA']
node_persons_array: 	 ['abdul rahman hussein']
node_organizations_array: 	 
----
node_id: 	 20200824224500-T1488
node_date: 	 20200824
node_themes_array: 	 ['UNGP_FORESTS_RIVERS_OCEANS', 'WB_137_WATER', 'AGRICULTURE', 'EPU_POLICY', 'EPU_POLICY_POLITICAL', 'USPEC_POLITICS_GENERAL1', 'NEGOTIATIONS', 'CRISISLEX_CRISISLEXREC', 'USPEC_POLICY1', 'EPU_ECONOMY', 'EPU_ECONOMY_HISTORIC', 'TAX_ETHNICITY', 'TAX_ETHNICITY_IRANIAN', 'WB_990_DRAINAGE', 'WB_151_IRRIGATION_AND_DRAINAGE', 'WB_140_AGRICULTURAL_WATER_MANAGEMENT', 'AFFECT', 'LEGISLATION', 'WB_135_TRANSPORT', 'WB_1174_WAREHOUSING_AND_STORAGE', 'WB_793_TRANSPORT_AND_LOGISTICS_SERVICES', 'WB_1973_FINANCIAL_RISK_REDUCTION', 'WB_435_AGRICULTURE_AND_FOOD_SECURITY', 'WB_337_INSURANCE', 'WB_1967_AGRICULTURAL_RISK_AN

In [16]:
# Knowing our graph characteristics...
len(G.nodes), len(G.edges)

(1167, 3782)

In [17]:
import random
from tqdm.notebook import tqdm

def regularization(G, dim, embedding_feature:str='embedding', iterations=15,mi=0.85):

  nodes = []

  # inicializando vetor f para todos os nodes
  for node in G.nodes():
    G.nodes[node]['f'] = np.array([0.0]*dim)
    if embedding_feature in G.nodes[node]: G.nodes[node]['f'] = G.nodes[node][embedding_feature]*1.0
    nodes.append(node)

  pbar = tqdm(range(0,iterations))

  for iteration in pbar:
    random.shuffle(nodes)
    energy = 0.0

    # percorrendo cada node
    for node in nodes:
      f_new = np.array([0.0]*dim)
      f_old = np.array(G.nodes[node]['f'])*1.0
      sum_w = 0.0

      # percorrendo vizinhos do onde
      for neighbor in G.neighbors(node):
          w = 1.0
          if 'weight' in G[node][neighbor]: w = G[node][neighbor]['weight']

          w /= np.sqrt(G.degree[neighbor])

          f_new += w*G.nodes[neighbor]['f']

          sum_w += w

      f_new /= sum_w
      

      G.nodes[node]['f'] = f_new*1.0

      if embedding_feature in G.nodes[node]:
        G.nodes[node]['f'] = G.nodes[node][embedding_feature]*mi + G.nodes[node]['f']*(1.0-mi)
      
      energy += np.linalg.norm(f_new-f_old)

    iteration+=1
    message = 'Iteration '+str(iteration)+' | Energy = '+str(energy)
    pbar.set_description(message)

  return G

In [18]:
G_regularized = regularization(G, 512, embedding_feature="embedding", iterations=5, mi=0.7)

### Constructing our graph and saving it in disk

In [19]:
# We'll relabel our nodes, since it's names are not convenient...
mapping = {value: idx for idx, value in enumerate(G_regularized.nodes())}
G_dataset = nx.relabel_nodes(G_regularized, mapping=mapping, copy=True)
G_dataset.edges()

EdgeView([(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 9), (1, 66), (1, 145), (1, 178), (1, 218), (1, 258), (1, 299), (1, 333), (1, 363), (1, 383), (1, 387), (1, 396), (1, 409), (1, 419), (1, 435), (1, 448), (1, 469), (1, 475), (1, 494), (1, 501), (1, 523), (1, 548), (1, 558), (1, 581), (1, 602), (1, 611), (1, 626), (1, 633), (1, 642), (1, 650), (1, 657), (1, 668), (1, 674), (1, 678), (1, 683), (1, 686), (1, 701), (1, 704), (1, 708), (1, 720), (1, 734), (1, 743), (1, 746), (1, 752), (1, 767), (1, 769), (1, 785), (1, 816), (1, 823), (1, 854), (1, 877), (1, 884), (1, 898), (1, 903), (1, 911), (1, 921), (1, 928), (1, 949), (1, 966), (1, 970), (1, 977), (1, 986), (1, 990), (1, 1002), (1, 1008), (1, 1029), (1, 1036), (1, 1047), (1, 1052), (1, 1054), (1, 1058), (1, 1067), (1, 1075), (1, 1094), (1, 1097), (1, 1099), (1, 1103), (1, 1104), (1, 1109), (1, 1111), (1, 1112), (1, 1117), (1, 1121), (1, 1123), (1, 1129), (1, 1134), (1, 1136), (1, 1140), (1, 1145), (1, 1147), (1

In [37]:
nx.write_gpickle(G_dataset, './test.gpickle')

In [38]:
G_read = nx.read_gpickle('./test.gpickle')

In [40]:
G_read.edges()

EdgeView([(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 9), (1, 66), (1, 145), (1, 178), (1, 218), (1, 258), (1, 299), (1, 333), (1, 363), (1, 383), (1, 387), (1, 396), (1, 409), (1, 419), (1, 435), (1, 448), (1, 469), (1, 475), (1, 494), (1, 501), (1, 523), (1, 548), (1, 558), (1, 581), (1, 602), (1, 611), (1, 626), (1, 633), (1, 642), (1, 650), (1, 657), (1, 668), (1, 674), (1, 678), (1, 683), (1, 686), (1, 701), (1, 704), (1, 708), (1, 720), (1, 734), (1, 743), (1, 746), (1, 752), (1, 767), (1, 769), (1, 785), (1, 816), (1, 823), (1, 854), (1, 877), (1, 884), (1, 898), (1, 903), (1, 911), (1, 921), (1, 928), (1, 949), (1, 966), (1, 970), (1, 977), (1, 986), (1, 990), (1, 1002), (1, 1008), (1, 1029), (1, 1036), (1, 1047), (1, 1052), (1, 1054), (1, 1058), (1, 1067), (1, 1075), (1, 1094), (1, 1097), (1, 1099), (1, 1103), (1, 1104), (1, 1109), (1, 1111), (1, 1112), (1, 1117), (1, 1121), (1, 1123), (1, 1129), (1, 1134), (1, 1136), (1, 1140), (1, 1145), (1, 1147), (1

### Playground to preprocessment (this section will be removed)

In [82]:
def process_event_dataset_from_networkx(G, features_attr="f", label_attr="themes", multi_label=False):
    """
    Builds an event graph dataset used in GAT model

    Parameters:
        G -> Graph representation of the event network (Networkx graph)
        features_att -> Feature attribute of each node (str)
        label_att -> Label attribute of each node (str)
        multi_label -> A boolean flag that considers a multi label dataset
    Returns:
        adj -> Sparse and symmetric adjacency matrix of our graph.
        features -> A NumPy matrix with our graph features.
        labels -> A NumPy array with the labels of all nodes.
        idx_train -> A NumPy array with the indexes of the training nodes.
        idx_val -> A NumPy array with the indexes of the validation nodes.
        idx_test -> A NumPy array with the indexes of the test nodes.

    """

    def encode_onehot_dict(labels):
        classes = set(labels)
        classes_dict = {c: i for i, c in enumerate(classes)}
        return classes_dict

    num_nodes = len(G.nodes)
    single_themes_dict = {}

    idxs = np.arange(0, num_nodes)
    np.random.shuffle(idxs)
    idx_train, idx_val, idx_test = np.split(
        idxs, [int(.6*num_nodes), int(.8*num_nodes)])
    

    # Organizing our feature matrix...
    # feature_matrix = np.array([ G.nodes[i]['embedding'] if 'embedding' in G.nodes[i].keys() else G.nodes[i][features_attr] for i in G.nodes()])
    feature_matrix = np.array([G.nodes[i][features_attr] for i in G.nodes()])
    feature_matrix = pd.DataFrame(feature_matrix, columns=[str(i) for i in range(feature_matrix.shape[1])])
    
    # Adding our labels to our feature_matrix dataframe
    if (multi_label):
        # TODO
        pass
    else:
        label_dict = nx.get_node_attributes(G, label_attr)
        single_label_dict = {i : label_dict[i][0] for i in label_dict.keys()}
        feature_matrix[label_attr] = pd.Series(single_label_dict)

        one_hot_dict = encode_onehot_dict(feature_matrix[label_attr].to_numpy())
        feature_matrix[label_attr] = feature_matrix[label_attr].map(one_hot_dict)

        
    feature_matrix = feature_matrix.to_numpy()

    labels = feature_matrix[:, -1]
    
    features = feature_matrix[:, :-1]

    adj = nx.adjacency_matrix(G)

    return adj, features, labels, idx_train, idx_val, idx_test

In [83]:
adj, features, labels, idx_train, idx_val, idx_test = process_event_dataset_from_networkx(G_dataset)

In [84]:
print(adj, features, labels, idx_train, idx_val, idx_test)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (1, 0)	1
  (1, 9)	1
  (1, 66)	1
  (1, 145)	1
  (1, 178)	1
  (1, 218)	1
  (1, 258)	1
  (1, 299)	1
  (1, 333)	1
  (1, 363)	1
  (1, 383)	1
  (1, 387)	1
  (1, 396)	1
  (1, 409)	1
  (1, 419)	1
  (1, 435)	1
  (1, 448)	1
  :	:
  (1160, 278)	1
  (1160, 279)	1
  (1160, 280)	1
  (1160, 281)	1
  (1160, 286)	1
  (1160, 287)	1
  (1160, 288)	1
  (1160, 289)	1
  (1160, 290)	1
  (1160, 505)	1
  (1160, 652)	1
  (1160, 653)	1
  (1160, 889)	1
  (1160, 1161)	1
  (1160, 1162)	1
  (1160, 1163)	1
  (1160, 1164)	1
  (1160, 1165)	1
  (1160, 1166)	1
  (1161, 1160)	1
  (1162, 1160)	1
  (1163, 1160)	1
  (1164, 1160)	1
  (1165, 1160)	1
  (1166, 1160)	1 [[-0.01688793 -0.03403207 -0.00421573 ... -0.00474867 -0.0454881
  -0.01568224]
 [-0.01413601 -0.03919175 -0.00138649 ... -0.01037473 -0.04606846
  -0.01194079]
 [-0.01447247 -0.0390778  -0.0012719  ... -0.00965512 -0.04551119
  -0.01207889]
 ...
 [-0.03921107  0.00955347  0.00

In [61]:
def encode_onehot_dict(labels):
    classes = set(labels)
    classes_dict = {c: i for i, c in enumerate(classes)}
    return classes_dict


In [67]:
encode_onehot_dict(["TESTE" for i in range(100)])

{'TESTE': 0}

In [31]:
type(adj)

scipy.sparse.csr.csr_matrix

In [ ]:
print(adj)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	1
  (0, 11)	1
  (1, 0)	1
  (1, 12)	1
  (1, 33)	1
  (1, 67)	1
  (1, 104)	1
  (1, 106)	1
  (1, 132)	1
  (1, 140)	1
  (1, 141)	1
  (1, 154)	1
  (1, 158)	1
  (1, 159)	1
  (1, 160)	1
  (1, 181)	1
  :	:
  (598, 20)	1
  (598, 21)	1
  (598, 22)	1
  (598, 23)	1
  (598, 24)	1
  (598, 25)	1
  (598, 37)	1
  (598, 38)	1
  (598, 39)	1
  (598, 59)	1
  (598, 65)	1
  (598, 122)	1
  (598, 133)	1
  (598, 198)	1
  (598, 199)	1
  (598, 200)	1
  (598, 201)	1
  (598, 205)	1
  (598, 206)	1
  (598, 341)	1
  (598, 400)	1
  (598, 409)	1
  (598, 420)	1
  (598, 599)	1
  (599, 598)	1
